In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from IPython.core.magic import register_cell_magic

from y2klc3tools.sqlvm import SqlVM
from y2klc3tools import UINT16_MAX, PC_START

In [ ]:
vm = SqlVM()


@register_cell_magic
def sql(_, cell):
    vm.run_and_trace(cell)

In [ ]:
vm.load_binary_from_file("../obj/vm/hello2.obj")
vm.reset()
# vm.continue_()

In [ ]:
from y2klc3tools.asm import assemble

code = """
.ORIG x3000
LEA R0, HELLO_WORLD
PUTS
HALT
HELLO_WORLD .stringz "Hello, World"
.END
"""

sym, image = assemble(code)
print(image)
vm.load_binary_from_bytes(image)
vm.reset()

In [ ]:
vm.step(trace=True)
vm.run_and_trace("""
    SELECT * FROM signal;
    SELECT * FROM register;
    SELECT * FROM memory
        WHERE address >= 0x3000 and address < 0x3040;
    """)

In [ ]:
%%sql
SELECT * FROM signal;
SELECT * FROM register;
SELECT * FROM memory
    WHERE address >= 0x3000 and address < 0x3040;

In [ ]:
origin = '0000'
image_bytes = origin + '16BF' * (UINT16_MAX)

vm.reset()
vm.load_binary_from_hex(image_bytes)

vm.memory[PC_START + 0x10] = 0xF025  # HLT

In [ ]:
# read a source code file using readlines

filename = "../README.md"

with open(filename) as f:
    lines = f.readlines()

print(lines)

In [ ]:
vm.step(trace=True)
vm.run_and_trace(
    """
    SELECT * FROM signal;
    SELECT * FROM register;
    SELECT * FROM memory
        WHERE address >= 0x3000 and address < 0x3040;
    """
)